In [2]:
from langchain.vectorstores import FAISS
from langchain.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain.memory import ConversationBufferMemory
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains.retrieval import create_retrieval_chain
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import CSVLoader
from langchain_groq import ChatGroq
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
import os
import tempfile
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
groq_api_key = os.getenv('GROQ_API_KEY')
pinecone_api_key = os.getenv('PINECONE_API_KEY')
os.environ['HF_TOKEN'] = os.getenv('HF_TOKEN')


In [4]:
loader = CSVLoader(file_path="ZOTA.csv")
data = loader.load()

In [5]:
splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=100)
splits = splitter.split_documents(data)

In [6]:
llm = ChatGroq(model="Llama-3.1-70b-Versatile", groq_api_key=groq_api_key)
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x7a960915d030>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x7a960915c370>, model_name='Llama-3.1-70b-Versatile', groq_api_key=SecretStr('**********'))

In [7]:
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
embeddings

/home/innobit/Desktop/work/financial_chatbot/finenv/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [8]:
vectordb = FAISS.from_documents(splits,embedding=embeddings)
vectordb

In [9]:
retriever = vectordb.as_retriever()

In [10]:
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)


In [11]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a financial analytics chatbot. Provide accurate and helpful answers based on your financial knowledge and the given context."),
    ("ai", "I understand. I'm here to assist with financial analysis and questions. How can I help you today?"),
    ("human", "{input}"),
    ("human", "Context: {context}"),
])

In [12]:
document_chain = create_stuff_documents_chain(llm, prompt)
retrieval_chain = create_retrieval_chain(retriever, document_chain)


In [16]:
input_text = "How is this stock performing?"
response = retrieval_chain.invoke({"input": input_text, "chat_history": memory.chat_memory.messages})
response['answer']

"Based on the provided data, I'll analyze the stock's performance.\n\n**Overall Trend:**\nThe stock's price has been declining over the past year. On 2019-09-16, the closing price was 197.05, and on 2020-11-06, it was 138.7, representing a decline of approximately 29.6%.\n\n**Recent Performance:**\nIn the recent past, the stock has been relatively stable. On 2020-05-27, the closing price was 139.85, and on 2020-06-16, it was 141.7, showing a slight increase of about 1.2%. However, on 2020-11-06, the closing price was 138.7, indicating a minor decline of about 2.6% from the 2020-06-16 price.\n\n**Volume and Turnover:**\nThe trading volume has been decreasing over time. On 2019-09-16, the volume was 22,918, while on 2020-11-06, it was 13,051, representing a decline of about 43%. The turnover has also decreased, from 451.46 billion on 2019-09-16 to 179.72 billion on 2020-11-06.\n\n**Deliverable Volume and Percentage:**\nThe deliverable volume has been increasing, from 532 on 2019-09-16 to